# Melhores Práticas de SQL e Performance de consultas 🚀

![Slow Query](../img/aula04/queries.jpg)

## Revisão Aula 03

![Grupos SQL](../img/aula01/grupos_sql.png)

Os tipos da linguagem SQL são:

- **DQL - Data Query Language** - Linguagem de Consulta de dados.
    - São os comandos de consulta. Exemplos: `SELECT`, `FROM`
- **DML - Data Manipulation Language** - Linguagem de Manipulação de Dados.
    - São os comandos que interagem com os dados dentro das tabelas. Exemplos: `INSERT`, `DELETE`, e `UPDATE`.
- DTL - Data Transaction Language - Linguagem de Transação de Dados.
    - São os comandos para controle de transação. Exemplos: `BEGIN TRANSACTION`, `COMMIT` e `ROLLBACK`

![UPDATE SEM WHERE](../img/aula03/update_sem_where_cert.png)

### Window Functions
![Window Functions](../img/aula04/window_functions.png)

### Pivot | Unpivot
![tDin](../img/aula03/pivot_table.png)

***

# Data matrix/tidy data principles 📋
It is best practice from an analyst's perspective for tables to be formatted in data matrix/tidy data format. For a table to be formatted in this way, it must adhere to two things:
- Each variable is a column
- Each observation is a row

It is best practice in the way that it standardises the way data is organised so the data cleaning process is easier and faster.

Whereas for messy, datasets, you can think of them like this:
> *Happy families are all alike; every unhappy family is unhappy in its own way* - Leo Tolstoy

From this persepctive, you can imagine that a messy dataset requires some initial upfront cost to understand how it is structured before you can clean it.

For a more thorough and example-laden discussion of tidy data principles, see this paper [here](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html)

> **TIP:** This principle applies more generally outside of SQL.

**Note:** The query below is quite difficult but think of it in the way it incorporates several things we learnt today, can you spot them?

In [9]:
-- create first table in messy format!
WITH table_messy_a AS 
(
    SELECT * 
    FROM 
    (
        VALUES
            ('Jane Smith', NULL, 18)
            ,('Xi Tang', 4, 1)
            ,('Park Min Woo', 6, 6)
    ) AS table_sub ([PersonName], [Treatment_a], [Treatment_b])
)
SELECT * 
FROM table_messy_a;

-- create second table in messy format!
WITH table_messy_b AS
(
    SELECT *
    FROM 
    (
        VALUES
            ('a', NULL, 4, 6)
            ,('b', 18, 1, 6)
    ) AS table_sub ([Treatment], [JaneSmith], [XiTang], [ParkMinWoo])
)
SELECT * 
FROM table_messy_b;

In [10]:
-- create above table in tidy format
WITH table_tidy AS
(
    SELECT *
    FROM
    (
        VALUES
            ('Jane Smith', 'a', NULL)
            ,('Jane SMith', 'b', 18)
            ,('Xi Tang', 'a', 4)
            ,('Xi Tang', 'b', 1)
            ,('Park Min Woo', 'a', 6)
            ,('Park Min Woo', 'b', 6)
    ) AS table_sub ([PersonName], [TreatmentType], [TreatmentValue])
)
SELECT *
FROM table_tidy;

*Aside: Whilst **data matrix/tidy data principles** are best practice for analysts, it is not for data architects working in SQL. Best practice for them would be to have tables in **long format**. Such a format enables total flexibility over table structure. This means when a new column needs to be added to a table, the table does not need to be deleted and created again with the new table (including the additional time required to import the data). Instead, such a format enables the additional column to be included as an extra row.* 

Further discussion of this is outside the scope of this training session, but if you want to find out more about this, then please read up on **snowflake schema** and **star schema**.

# Monitorar e ajustar desempenho 🚀

## Como as consultas são executadas ?

O *core* do SQL Server se baseia principalmente em dois componentes: o ***Storage Engine*** e o ***Query Processor***. O primeiro é responsável pelas operações de leitura/escrita de dados entre disco e memória de maneira otimizada. Já o segundo, como o nome sugere, recebe e analisa as consultas SQL, **gera um plano para sua execução otimizada** e retorna os resultados requisitados. 

![Query Optimizer](../img/aula04/query_optimizer.jpg)

O plano de execução de uma consulta SQL é uma representação gráfica, textual ou em XML, que mostra **quais operadores e operações** que o otimizador de consultas fez para retornar os dados da sua query, além do custo de cada operação realizada.

### Analisando o plano de execução (*Query Execution Plan*) de uma consulta

Para ativar a visualização do plano de execução da consulta, devemos utilizar a função ***Enable Actual Plan***, do Azure Data Studio ou ***Include Actual Execution Plan (Ctrl + M)***, do SSMS.

In [ ]:
-- Exemplo query Aula 01: Criando novas colunas

SELECT
    Station
    ,sample_id      AS [ID Amostra]
    ,sample_date    AS [Data Amostra]
    ,sample_type    AS [Tipo Amostra]
    ,CASE
        WHEN sample_date <= '2019-01-01' THEN 'Pré Instalação'
        WHEN sample_date <= '2022-05-01' THEN 'Durante Instalação'
        -- Condições adicionais...
        ELSE 'Pós Instalação'
    END             AS [Período]
FROM parameter_sample

### Exercício 1: 
Vamos analisar os desempenhos das consultas propostas como soluções para o Desafio da Aula 01 (Cálculo da vazão operacional de poços)...

> Dica: Podemos adicionar a instrução `SET STATISTICS IO, TIME ON;` antes da consulta, para gerar estatísticas adicionais.

1. Desconsiderando os campos **Hidrometro Inicial** e **Horimetro Inicial**:

In [ ]:
-- Solução 01: Subqueries

SELECT
	wo.Station
	,wo.zdate_                      AS [Data]
	,wo.previous_date				AS [Ultima Data]
	,wo.zhourmeter_final 			AS [Horimetro]
	,wo.previous_hourmeter 	 		AS [Ultimo Horimetro]
	,wo.zhydrometer_final 			AS [Hidrometro]
	,wo.previous_hydrometer 		AS [Ultimo Hidrometro]

	-- Volume e Tempo operacional calculados
	,(wo.zhydrometer_final - wo.previous_hydrometer)				AS [Volume (m3)]
	,ROUND((wo.zhourmeter_final - wo.previous_hourmeter), 2)		AS [Horas Operacao (h)]

	-- Vazão calculada
	,ROUND(
		(wo.zhydrometer_final - wo.previous_hydrometer) / 
		NULLIF(wo.zhourmeter_final - wo.previous_hourmeter, 0) -- Tratar erro de divisão por 0
	,3)								AS [Vazao (m3/h)]

	,wo.zfrequency					AS [Frequencia]
	,wo.[zdata_type]                AS [Tipo Dado]
	,wo.[zstatus]                   AS [Status]
FROM ( 
	 SELECT 
        -- Retorna todos os campos da tabela well_operation (outer)
	 	wo_outer.*,

        -- Retorna a data mais recente (TOP 1 ordenando pela Data)
        -- de uma nova tabela well_operation (inner).
        -- Função WHERE filtra sempre o mesmo ponto e apenas datas anteriores
		( 
			SELECT TOP 1 wo_inner.[zdate_]
			FROM [zwell_operation] wo_inner
			WHERE wo_outer.[zdate_] > wo_inner.[zdate_] AND wo_outer.[Station] = wo_inner.[Station]
			ORDER BY wo_inner.zdate_ DESC
		)                                       AS [previous_date],

        -- Mesmo padrão para o registro mais recente,
        -- mas desta vez retornando o campo [zhourmeter_final]
		( 
			SELECT TOP 1 wo_inner.[zhourmeter_final]
			FROM [zwell_operation] wo_inner
			WHERE wo_outer.[zdate_] > wo_inner.[zdate_] AND wo_outer.[Station] = wo_inner.[Station]
			ORDER BY wo_inner.zdate_ DESC
		)                                       AS [previous_hourmeter],

        -- Mesmo padrão para o registro mais recente,
        -- mas desta vez retornando o campo [zhydrometer_final]
		( 
			SELECT TOP 1 wo_inner.[zhydrometer_final]
			FROM [zwell_operation] wo_inner
			WHERE wo_outer.[zdate_] > wo_inner.[zdate_] AND wo_outer.[Station] = wo_inner.[Station]
			ORDER BY wo_inner.zdate_ DESC
		)                                       AS [previous_hydrometer]
	FROM [zwell_operation] wo_outer
) wo

In [ ]:
-- Solução 02: CTE + ROW_NUMBER()

-- Enumera os registros como 1..N para cada Station, ordenando por data
WITH cte AS (
    SELECT Station, zdate_, zhydrometer_final, zhourmeter_final, zfrequency, zdata_type, zstatus,
        ROW_NUMBER() OVER (PARTITION BY Station ORDER BY zdate_ ASC) rn
    FROM zwell_operation
)

SELECT
	a.Station
	,a.zdate_                       AS [Data]
	,b.zdate_					    AS [Ultima Data]
	,a.zhourmeter_final 			AS [Horimetro]
	,b.zhourmeter_final 			AS [Ultimo Horimetro]
	,a.zhydrometer_final 			AS [Hidrometro]
	,b.zhydrometer_final 			AS [Ultimo Hidrometro]

	-- Volume e Tempo operacional calculados
	,(a.zhydrometer_final - b.zhydrometer_final)		    AS [Volume (m3)]
	,ROUND((a.zhourmeter_final - b.zhourmeter_final), 2)	AS [Horas Operacao (h)]

	-- Vazão calculada
	,ROUND(
        (a.zhydrometer_final - b.zhydrometer_final) / 
        NULLIF(a.zhourmeter_final - b.zhourmeter_final, 0) -- Tratar erro de divisão por 0
    ,3)	                            AS [Vazao (m3/h)]

	,a.zfrequency                   AS [Frequencia]
	,a.[zdata_type]                 AS [Tipo Dado]
	,a.[zstatus]                    AS [Status]

FROM cte a
    LEFT JOIN cte b
        ON a.Station = b.Station
        AND a.rn = b.rn + 1

In [ ]:
-- Solução 03: LAG

SELECT
	wo.Station
	,wo.zdate_                      AS [Data]
	,wo.previous_date				AS [Ultima Data]
	,wo.zhourmeter_final 			AS [Horimetro]
	,wo.previous_hourmeter 	 		AS [Ultimo Horimetro]
	,wo.zhydrometer_final 			AS [Hidrometro]
	,wo.previous_hydrometer 		AS [Ultimo Hidrometro]

	-- Volume e Tempo operacional calculados
	,(wo.zhydrometer_final - wo.previous_hydrometer)				AS [Volume (m3)]
	,ROUND((wo.zhourmeter_final - wo.previous_hourmeter), 2)		AS [Horas Operacao (h)]

	-- Vazão calculada
	,ROUND(
		(wo.zhydrometer_final - wo.previous_hydrometer) / 
		NULLIF(wo.zhourmeter_final - wo.previous_hourmeter, 0) -- Tratar erro de divisão por 0
	,3)								AS [Vazao (m3/h)]

	,wo.zfrequency					AS [Frequencia]
	,wo.[zdata_type]                AS [Tipo Dado]
	,wo.[zstatus]                   AS [Status]

FROM (
    SELECT
        Station
        ,zdate_
        ,LAG(zdate_) OVER (PARTITION BY Station ORDER BY zdate_ ASC) AS previous_date
        ,zhydrometer_final
        ,LAG(zhydrometer_final) OVER (PARTITION BY Station ORDER BY zdate_ ASC) AS previous_hydrometer
        ,zhourmeter_final
        ,LAG(zhourmeter_final) OVER (PARTITION BY Station ORDER BY zdate_ ASC) AS previous_hourmeter
        ,zfrequency
        ,zdata_type
        ,zstatus
    FROM zwell_operation
) wo

2. Considerando os campos **Hidrometro Inicial** e **Horimetro Inicial**:

In [ ]:
-- Solução inicial

SELECT
	wo.Station
	,wo.zdate_                      AS [Data]
	,wo.zhourmeter_start 			AS [Horimetro Inicial]
	,wo.zhourmeter_final 			AS [Horimetro Final]
	,wo.zhydrometer_start 			AS [Hidrometro Inicial]
	,wo.zhydrometer_final 			AS [Hidrometro Final]
    ,wo.zoperation_hours            AS [Horas Operacao (h)]
	,wo.zoperation_volum            AS [Volume (m3)]	

    -- Calcular Vazão, de acordo com dados disponíveis
    ,ROUND(
        COALESCE(
            wo.zflowrate,               -- Retornar caso seja diferente de NULL
            COALESCE(                   -- Numerador = Volume (caso não nulo) ou a diferença de leitura dos hidrômetros
                wo.zoperation_volum,
                wo.zhydrometer_final - wo.zhydrometer_start
            ) / 
            COALESCE(                   -- Denominador = Horas operação (caso não nulo) ou a diferença de leitura dos horímetros
                NULLIF(wo.zoperation_hours, 0),
                NULLIF(wo.zhourmeter_final - wo.zhourmeter_start, 0) -- Tratar erro de divisão por 0
            )
        )
	,3)								AS [Vazao (m3/h)]

	,wo.zfrequency					AS [Frequencia]
	,wo.[zdata_type]                AS [Tipo Dado]
	,wo.[zstatus]                   AS [Status]
FROM zwell_operation wo

#### Primeiras conclusões:

- Ordem de execução das etapas;
- Linguagem SQL tem natureza altamente declarativa;
  - Exige apenas instruções de quais dados obter, não o algoritmo completo para executar.
- Impacto de decisões de modelagem de dados no desempenho das consultas

## Exercício 2:

Vamos comparar as consultas propostas como soluções para o desafio da Aula 03... Quais conclusões podemos chegar ?

In [ ]:
-- 

> Na maioria das vezes, o *Query Optimizer* faz um ótimo trabalho na escolha de planos de execução altamente eficientes, inclusive recomendando a criação de índices específicos que podem melhorar a performance de uma consulta.

Existem diversas técnicas e ferramentas mais robustas para a otimização de consultas, indo desde *tracers* e monitoramento de atividade até o uso de IA e *machine learning*, de acordo com o sistema e a plataforma.

## Boas práticas

![AI SQL](../img/aula04/ai_queries.png)

De maneira geral, podemos listar algumas boas práticas para desenvolver consultas SQL melhores, considerando não apenas o desempenho. 💡

### 1. Precisão, legibilidade e otimização: nessa ordem

- Evite ajustar consultas SQL até garantir que ela retorna os dados desejados, ou seja, que a **lógica da consulta está correta**.
- Priorize a otimização das consultas executadas com frequência ou as que envolvem tabelas com maior quantidade de linhas.
- Priorize a legibilidade, ou seja, a facilidade para outras pessoas entenderem e modificarem o código, antes de se preocupar com o desempenho.

#### 1.1 Comente o código (por favor 🙏), principalmente as regras de negócio envolvidas.
- Ajude ao próximo (e a ti mesmo)

In [ ]:
-- Exemplo:
-- O código a seguir obviamente filtra as medições mensais a partir de 2023
-- Mas porque ? Qual a regra de negócio envolvida ? 

SELECT
    Station
    ,date_                              AS [DataHora]
    ,CAST(date_ AS DATE)                AS [Data]
    ,FORMAT(date_, 'MMM/yy')            AS [MesAno]
    ,depth_                             AS [Profundidade (m)]
    ,zwl_elevation                      AS [Cota NA (m)]
    ,dry_indicator                      AS [Seco?]
    ,zstatus                            AS [Status]
FROM gw_level
-- Filtra 2023
WHERE [date_] >= '2023-01-01' AND zfrequency = 'Mensal'

In [ ]:
/* 
Consulta para retornar medições mensais de 2023
Desenvolvido em 2023
Autor: WST
 */

SELECT Station
       ,date_                              AS [DataHora]
       ,CAST(date_ AS DATE)                AS [Data]
       ,FORMAT(date_, 'MMM/yy')            AS [MesAno]
       ,depth_                             AS [Profundidade (m)]
       ,zwl_elevation                      AS [Cota NA (m)]
       ,dry_indicator                      AS [Seco?]
       ,zstatus                            AS [Status]
FROM gw_level
WHERE [date_] >= '2023-01-01'
    AND zfrequency = 'Mensal'

#### 1.2 Formate a consulta e utilize aliases

- Uma consulta bem formatada sempre melhora significativamente a legibilidade do código.
- Abuse de espaços, tabulações e identações para tornar o código mais legível.
- Utilize letras MAIÚSCULAS para as palavras reservadas, como `SELECT`, `FROM` e `WHERE`.

> Podemos utilizar ferramentas de formatação de SQL online ou extensões.
> [SQL Style Guide](https://www.sqlstyle.guide/)

In [ ]:
SELECT Station
       ,date_                              AS [DataHora]
       ,CAST(date_ AS DATE)                AS [Data]
       ,FORMAT(date_, 'MMM/yy')            AS [MesAno]
       ,depth_                             AS [Profundidade (m)]
       ,zwl_elevation                      AS [Cota NA (m)]
       ,dry_indicator                      AS [Seco?]
       ,zstatus                            AS [Status]
  FROM gw_level
 WHERE [date_] >= '2023-01-01' 
   AND zfrequency = 'Mensal'

### 2. Evite usar `SELECT *`

- Selecione apenas as colunas necessárias para atender aos objetivos da consulta.
- Para visualizar o formato de tabelas durante o desenvolvimento de uma consulta SQL, utilize o comando `TOP`.
  
> Lembre-se: Quanto menor a quantidade de linhas lidas, melhor o desempenho da consulta...

### 3. Lembre-se da ordem de execução das consultas

![Ordem SQL](../img/aula01/execution_order.png)

- Quanto antes for possível limitar o conjunto de dados a ser processado, melhor.

### 4. Evite usar funções na cláusula WHERE

In [ ]:
-- ❌ Ao invés de
SELECT Station, date_, total_precipitation
  FROM dbo.meteorology
  WHERE YEAR(date_) = 2022

-- ✅ É melhor utilizar
SELECT Station, date_, total_precipitation
  FROM dbo.meteorology
  WHERE date_ BETWEEN '2022-01-01' AND '2022-12-31' 

#### 4.1. Evite usar o operador `LIKE` na cláusula WHERE

- O operador `LIKE` compara caracteres e permite o uso de caracteres curinga como `%`, mas pode apresentar lentidão.
- Quando necessário, evite utilizar caracteres como o `%` **no início** das strings.

In [ ]:
-- ❌ Ao invés de
SELECT * FROM station WHERE mine LIKE '%Barragem%'

-- ✅ É melhor utilizar
SELECT * FROM station WHERE mine LIKE 'Barragem%'

SELECT * FROM station WHERE mine = 'Barragem A' OR mine = 'Barragem B'
SELECT * FROM station WHERE mine IN ('Barragem A', 'Barragem B')

### 5. Evite usar a instrução `ORDER BY`, especialmente em sub-queries

- Ordenar registros é uma operação extremamente custosa.
- A menos que estritamente necessário, opte por ordenar os resultados na ferramenta de análise, não na consulta SQL

### 6. Faça escolhas de JOINs inteligentes

- Utilize JOINs apenas quando necessário.
- Prefira a estrutura `<tabela_maior> JOIN <tabela_menor>` ao invés de `<tabela_menor> JOIN <tabela_maior>`.
- Use os operadores `=` e `AND` nas condições do `JOIN`; evite usar os operadores `OR` e `<>`.
- Prefira `INNER JOIN` ou `LEFT/RIGHT [OUTER] JOIN` ao invés do `CROSS JOIN` !

📚 Recomendações finais:

- [Tidy Data Definition](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html)
- [SQL *vs* Pandas cheatsheet](https://sql2pandas.pythonanywhere.com/download-files/sql-to-pandas-cheat-sheet)
- [SQL Server Query Performance Guidelines Tutorial](https://www.mssqltips.com/sqlservertutorial/3200/sql-server-query-performance-guidelines-tutorial/)